Open the bash container:

In [ ]:
img_path=../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

Generate fragments:

In [4]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8

subset=UCS
for file in full_preprocessing_out/data/reports/barcard/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.tomerge.tsv}
    # echo $sample
    jaccardfile=$file
    infile=full_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=full_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    elif [ -f "$outfile2" ]; then
        echo "$outfile2 exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
    fi
done

full_preprocessing_out/data/reports/barcard/BIO_ddseq_1.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_1.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_2.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_2.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_3.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_3.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_4.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_4.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BRO_mtscatac_1.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_1.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BRO_mtscatac_2.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_2.FULL.fragments.tsv.gz exists, skipping
full_preprocessi

Then bgzip the output:

In [2]:
module load HTSlib
subset=
for file in full_preprocessing_out/data/fragments/*$subset*.fragments.tsv
do
    echo $file
    if [ -f "$file.gz" ]; then
        echo exists
    else
        bgzip -@ 8 $file &
    fi
done

full_preprocessing_out/data/fragments/OHS_s3atac_1.FULL.fragments.tsv
exists
full_preprocessing_out/data/fragments/OHS_s3atac_2.FULL.fragments.tsv
exists


and tabix index

In [ ]:
module load HTSlib
subset=
for file in full_preprocessing_out/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done

And add a new barcode tag `DB` to the corresponding .bam files, based on the barcode translate files

In [2]:
cat full_preprocessing_out/data/fragments/CNA_10xv11_1.FULL.fragments.tsv.original_CB_to_merged_CBs.tsv

original_CB	merged_CBs
AGCGTGCTCGATGCAT	AGCGTGCTCGATGCAT_CTCAGCTCACGATATC
GGTGTCGTCCATTGTT	GGTGTCGTCCATTGTT_TTCATTGGTTGGCGAA
TTAACGGCACTACACA	GAACCGCAGTTGTCCA_TTAACGGCACTACACA
ACTGCAAAGGATGTCG	ACTGCAAAGGATGTCG_GTGACATTCGAGGCTC
CTCAGCTCACGATATC	AGCGTGCTCGATGCAT_CTCAGCTCACGATATC
CCACGTTTCTTCAGAG	CCACGTTTCTTCAGAG_CCATACCGTTTAGGAA
CTGGGACAGCACACCC	CTGGGACAGCACACCC_GCCATAAAGCTGCTCG
TGTGACAGTTTGTACG	CTCGCTACACCCTTAC_TGTGACAGTTTGTACG
ACTGCAACATTAGCCA	ACGGATTAGTATCTGC_ACTGCAACATTAGCCA
CGCTATCCAAATTGAG	CGCTATCCAAATTGAG_GAATCTGTCGATAGGG
CGAGTTACATCAGAAA	CGAGTTACATCAGAAA_GCTGAGCGTCCGGTAC
CCGCATTAGCTCCATA	AGTCAACCAGAACTTC_CCGCATTAGCTCCATA
CGTTCCAGTCCCGTGA	CGTTCCAGTCCCGTGA_GCCAGCAGTTTGTACG
TACGGATTCGTAGCGC	ATTCGTTGTTAGGAGC_TACGGATTCGTAGCGC
AACCGATCATTCGTCC	AACCGATCATTCGTCC_GCGAGTCTCTAGCAAC
GCAGCCACAGGATAGC	CCACAGGCAAATTGAG_GCAGCCACAGGATAGC
TGTGTCCAGCACGTAG	ACTGTCCGTACTTGAC_TGTGTCCAGCACGTAG
GCCATAAAGCTGCTCG	CTGGGACAGCACACCC_GCCATAAAGCTGCTCG
GATCGTAGTTCTCGAA	GATCGTAGTTCTCGAA_TACGGATTCTGGCACG
GTCGTAAA

In [7]:
prog=../0_resources/single_cell_toolkit_rust/target/release/add_new_bc_tag
prog=/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/0_resources/single_cell_toolkit_rust/add_new_bc_tag_if_old_found
for file in full_preprocessing_out/data/bam/UCS*possorted.bam
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%_____R1.bwa.out.possorted.bam}
    echo $sample
    outfile=full_preprocessing_out/data/bam/$sample.possorted.dbfixed.bam
    bc_map=full_preprocessing_out/data/fragments/$sample.fragments.tsv.original_CB_to_merged_CBs.tsv
    ls $bc_map
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        $prog $file $outfile $bc_map CB DB &
    fi
done

full_preprocessing_out/data/bam/BIO_ddseq_1.FULL_____R1.bwa.out.possorted.bam
BIO_ddseq_1.FULL
full_preprocessing_out/data/fragments/BIO_ddseq_1.FULL.fragments.tsv.original_CB_to_merged_CBs.tsv
full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam does not exist, generating!



In [2]:
for file in full_preprocessing_out/data/bam/*.possorted.dbfixed.bam
do
    echo $file
    outfile=$file.bai
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        samtools index -@ 8 $file &
    fi
done

full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam.bai does not exist, generating!
full_preprocessing_out/data/bam/BIO_ddseq_2.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_2.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BIO_ddseq_3.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_3.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BIO_ddseq_4.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_4.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BRO_mtscatac_1.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BRO_mtscatac_1.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BRO_mtscatac_2.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BRO_mtscatac_2.FULL.possorted.dbfixed.bam.bai exists, skipping